In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import tree

In [2]:
from sqlalchemy import create_engine
from hold import connection_string

In [3]:
engine = create_engine(f'{connection_string}', encoding='iso-8859-1', connect_args={'connect_timeout': 10})

In [4]:
gtdDF = pd.read_sql_table('global_terrorism', con= engine)

In [5]:
# Need to drop NA's specific to provstate column
gtd_ProvinceDF = gtdDF.dropna(subset=['provstate'])
gtd_ProvinceDF.head()

,iyear,imonth,iday,date,country_txt,region_txt,provstate,city,latitude,longitude,...,claimmode_txt,weaptype1_txt,weapsubtype1_txt,weaptype2_txt,weapsubtype2_txt,kidhijcountry,ransom,nkill,nwound,index1
0,2000,1,1,2000-01-01,Namibia,Sub-Saharan Africa,Kavango,Muitjiku,-17.91,19.99,...,None,Firearms,Unknown Gun Type,None,None,None,NaN,0.0,7.0,1.0
1,2000,1,1,2000-01-01,Namibia,Sub-Saharan Africa,Kavango,Muitjiku,-17.91,19.99,...,None,Firearms,Unknown Gun Type,None,None,None,0.0,0.0,7.0,2.0
2,2000,1,1,2000-01-01,India,South Asia,Jammu and Kashmir,Udhampur,32.93,75.14,...,None,Firearms,Unknown Gun Type,None,None,None,NaN,1.0,2.0,3.0
3,2000,1,1,2000-01-01,Kosovo,Eastern Europe,Kosovo (Province),Peje,42.66,20.31,...,None,Explosives/Bombs/Dynamite,Unknown Explosive Type,None,None,None,NaN,0.0,1.0,4.0
4,2000,1,1,2000-01-01,Somalia,Sub-Saharan Africa,Banaadir,Mogadishu,2.11,45.39,...,None,Firearms,Unknown Gun Type,None,None,None,NaN,6.0,10.0,5.0


In [6]:
casualties = gtdDF['nkill'] + gtdDF['nwound']
casualties.sum()

554384.0

In [7]:
gtd_ProvinceDF['casualties'] = gtd_ProvinceDF['nkill'] + gtd_ProvinceDF['nwound']
gtd_ProvinceDF.head()

C:\Users\kruiz\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,iyear,imonth,iday,date,country_txt,region_txt,provstate,city,latitude,longitude,...,weaptype1_txt,weapsubtype1_txt,weaptype2_txt,weapsubtype2_txt,kidhijcountry,ransom,nkill,nwound,index1,casualties
0,2000,1,1,2000-01-01,Namibia,Sub-Saharan Africa,Kavango,Muitjiku,-17.91,19.99,...,Firearms,Unknown Gun Type,None,None,None,NaN,0.0,7.0,1.0,7.0
1,2000,1,1,2000-01-01,Namibia,Sub-Saharan Africa,Kavango,Muitjiku,-17.91,19.99,...,Firearms,Unknown Gun Type,None,None,None,0.0,0.0,7.0,2.0,7.0
2,2000,1,1,2000-01-01,India,South Asia,Jammu and Kashmir,Udhampur,32.93,75.14,...,Firearms,Unknown Gun Type,None,None,None,NaN,1.0,2.0,3.0,3.0
3,2000,1,1,2000-01-01,Kosovo,Eastern Europe,Kosovo (Province),Peje,42.66,20.31,...,Explosives/Bombs/Dynamite,Unknown Explosive Type,None,None,None,NaN,0.0,1.0,4.0,1.0
4,2000,1,1,2000-01-01,Somalia,Sub-Saharan Africa,Banaadir,Mogadishu,2.11,45.39,...,Firearms,Unknown Gun Type,None,None,None,NaN,6.0,10.0,5.0,16.0


In [16]:
ProvinceCasualtiesDF = gtd_ProvinceDF[['index1','provstate','casualties']]
ProvinceCasualtiesDF = ProvinceCasualtiesDF.reset_index()
ProvinceCasualtiesDF.head()

,index,index1,provstate,casualties
0,0,1.0,Kavango,7.0
1,1,2.0,Kavango,7.0
2,2,3.0,Jammu and Kashmir,3.0
3,3,4.0,Kosovo (Province),1.0
4,4,5.0,Banaadir,16.0


In [17]:
gtd_ProvinceDF['attacktype1_txt'].count()

92646

In [18]:
# Multiply the dummies by the casualties so that the regression works for casualties
ProvinceCasualties2DF = pd.get_dummies(ProvinceCasualtiesDF['provstate']).mul(ProvinceCasualtiesDF['casualties'],0)
ProvinceCasualties2DF = ProvinceCasualties2DF.reset_index()
#CountryCasualties2DF = pd.get_dummies(CountryCasualtiesDF['country_txt']).mul(CountryCasualtiesDF['casualties'],0)
#CountryCasualties2DF = CountryCasualties2DF.reset_index()
#RegionCasualties2DF = RegionCasualties2DF.rename(columns = {"index":"index1"})
ProvinceCasualties2DF.head()

,index,(Region) of Republican Subordination (Province),(Republic of) Ingushetia,Abia,Abkhazia,Abkhazia (Autonomous Region),Abkhazia (Autonomous Republic),Abkhazia (Region),Abra,Abruzzo,...,Zubin Potok,Zug,Zugdidi,Zulia,Zurich,khon kaen,phayao,sakon nakhon,samut prakan,sulawesi utara
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
ProvinceCasualtiesMergeDF = result = pd.merge(ProvinceCasualties2DF,ProvinceCasualtiesDF[['index','provstate','casualties']],on='index')
ProvinceCasualtiesMergeDF.head()

,index,(Region) of Republican Subordination (Province),(Republic of) Ingushetia,Abia,Abkhazia,Abkhazia (Autonomous Region),Abkhazia (Autonomous Republic),Abkhazia (Region),Abra,Abruzzo,...,Zugdidi,Zulia,Zurich,khon kaen,phayao,sakon nakhon,samut prakan,sulawesi utara,provstate,casualties
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Kavango,7.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Kavango,7.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Jammu and Kashmir,3.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Kosovo (Province),1.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Banaadir,16.0


In [21]:
# Assign X (data) and y (target)
data = ProvinceCasualtiesMergeDF.drop(["index","provstate",'casualties'], axis=1)
target = ProvinceCasualtiesMergeDF["provstate"]
feature_names = data.columns

print(data.shape, target.shape)

(92592, 1925) (92592,)


In [22]:
# Split the data into training and testing
### BEGIN SOLUTION
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)
### END SOLUTION

In [23]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.06704682909970623